Loading Libraries

In [18]:
import numpy as np
import gensim
import pandas as pd
from sklearn import model_selection, preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Loading Dataset

In [19]:
df = pd.read_csv('Dataset.csv')

Finding the maximum length of our tweets

In [20]:
df['word_count'] = df['clean'].apply(lambda x: len(str(x).split()))
max(df['word_count'])

45

Spliting the dataset into 70% training, 30% testing. Notic that we shuffled the data so it won't have a sprcific order.

In [21]:
# split the dataset into training and test datasets 70% training, 30% testing
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['clean'], df['sentiment'], test_size=0.3, random_state = 1)

Encoding the labels using preprocessing.LabelEncoder

In [23]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

Building the tokenizer to convert the tweets into sequences and pad them to the same length

In [24]:
#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(train_x))

#converting text into integer sequences
train_x_seq  = tokenizer.texts_to_sequences(train_x) 
test_x_seq = tokenizer.texts_to_sequences(test_x)

#padding to prepare sequences of same length
train_x_seq  = pad_sequences(train_x_seq, maxlen=45)
test_x_seq = pad_sequences(test_x_seq, maxlen=45)

In [25]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

81074


# Word Embeddings Using Pre-Traind Model

We will use the pre-trained AraVec Model. AraVec Model converts words into 300 dimensional vectors

In [26]:
# load the AraVec model
trained_model = gensim.models.Word2Vec.load("tweet_cbow_300/tweets_cbow_300")
print("We've",len(trained_model.wv.index2word),"vocabularies")

We've 331679 vocabularies


Loading Word Embeddings

In [45]:
# create a weight matrix for words in training docs
out_of_vocab = []
embedding_matrix = np.zeros((size_of_vocabulary, 300))
for index, word in enumerate(tokenizer.word_index):
    if word in trained_model.wv:
        embedding_vector = trained_model.wv[word]
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    else:
        out_of_vocab.append(word)

In [47]:
len(set(out_of_vocab))

27777

Building the model. We will use LSTM model

In [48]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix],input_length=45,trainable=True)) 

#lstm layer
model.add(LSTM(128))


#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 45, 300)           24322200  
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_15 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 24,550,169
Trainable params: 24,550,169
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
history = model.fit(np.array(train_x_seq),np.array(train_y),batch_size=256,epochs=10,
                    validation_data=(np.array(test_x_seq),np.array(test_y)),callbacks=[es,mc])

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 29374 samples, validate on 12589 samples
Epoch 1/10
29374/29374 [==============================] - 85s 3ms/step - loss: 0.5585 - acc: 0.6322 - val_loss: 0.4207 - val_acc: 0.6734

Epoch 00001: val_acc improved from -inf to 0.67345, saving model to best_model.h5
Epoch 2/10
29374/29374 [==============================] - 101s 3ms/step - loss: -0.3342 - acc: 0.7062 - val_loss: -0.6500 - val_acc: 0.6682

Epoch 00002: val_acc did not improve from 0.67345
Epoch 3/10
29374/29374 [==============================] - 101s 3ms/step - loss: -9.2545 - acc: 0.7296 - val_loss: -5.7452 - val_acc: 0.6958

Epoch 00003: val_acc improved from 0.67345 to 0.69577, saving model to best_model.h5
Epoch 4/10
29374/29374 [==============================] - 97s 3ms/step - loss: -39.5739 - acc: 0.7441 - val_loss: -12.4228 - val_acc: 0.6852

Epoch 00004: val_acc did not improve from 0.69577
Epoch 5/10
29374/29374 [==============================] - 101s 3ms/step - loss: -115.5334 - acc: 0.7705 - val_loss: -27.6

In [50]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(test_x_seq,test_y, batch_size=128)
print(val_acc)

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


12589/12589 [==============================] - 5s 432us/step
0.6957661509513855


# Word Embeddings From Scratch

In [42]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=45,trainable=True)) 

#lstm layer
model.add(LSTM(128))


#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min',patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True)  

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 45, 300)           24322200  
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 24,550,169
Trainable params: 24,550,169
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
history = model.fit(np.array(train_x_seq),np.array(train_y),batch_size=256,epochs=10,
                    validation_data=(np.array(test_x_seq),np.array(test_y)),callbacks=[es,mc])

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 29374 samples, validate on 12589 samples
Epoch 1/10
29374/29374 [==============================] - 102s 3ms/step - loss: 0.3170 - acc: 0.6817 - val_loss: -0.4927 - val_acc: 0.7114

Epoch 00001: val_acc improved from -inf to 0.71141, saving model to best_model.h5
Epoch 2/10
29374/29374 [==============================] - 105s 4ms/step - loss: -7.9946 - acc: 0.7754 - val_loss: 5.7221 - val_acc: 0.6853

Epoch 00002: val_acc did not improve from 0.71141
Epoch 3/10
29374/29374 [==============================] - 103s 3ms/step - loss: -43.8783 - acc: 0.8164 - val_loss: 2.3317 - val_acc: 0.6844

Epoch 00003: val_acc did not improve from 0.71141
Epoch 4/10
29374/29374 [==============================] - 102s 3ms/step - loss: -104.7294 - acc: 0.8368 - val_loss: -6.6938 - val_acc: 0.6906

Epoch 00004: val_acc did not improve from 0.71141
Epoch 5/10
29374/29374 [==============================] - 103s 3ms/step - loss: -129.9116 - acc: 0.8350 - val_loss: -0.8025 - val_acc: 0.6655

Epoch 00005

In [44]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(test_x_seq,test_y, batch_size=128)
print(val_acc)

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


12589/12589 [==============================] - 6s 488us/step
0.7114147543907166
